# S&P 2019 Data Wrangling

**by Marc Angelo Acebedo**

## Table of Contents
- [Introduction](#introduction)
- [Data Wrangling](#wrangling)
    - [Gather](#gather)
    - [Assess](#assess)
    - [Clean](#clean)
    - [Store](#store)

## I) Introduction <a id = "introduction">

**Broad question:** How do price forecasts for each firm in the S&P 2019 Index compare to their corresponding actual prices?

**Approach:** Analyze difference in means between average forecast EPS and average actual EPS for each firm.

I will be analyzing quarterly price returns within the past 20 years for the firms present in the S&P 500 2019 Index.

> At first, I wanted to analyze the forecasted vs. actual price earnings of the S&P in its entirety for the past 20 years. However, considering that firms continuously enter and leave stock indices every year, there would be varying levels of inconsistencies and marginal errors when comparing annual S&P returns alone. To combat this problem, I have isolated these two approaches:
- Analyze the historical earnings of *only* the firms present in the S&P 2019 Index
- Keep track of all firms that were present in the S&P for the past 20 years. Keep track of how many times each firm appeared in the Index and for those with the least count, analyze them individually on how they differ from the firms that stayed for longer.

[TK] HEre is a breakdown of my final clean CSV's features, TK.csv.

## II) Data Wrangling <a id="wrangling"></a>

To gather the data depicted under the `./data` folder, I used Bloomberg Excel functions.

### A) Gather <a id = "gather"></a>
> **APPROACH 1:** Focus on the firms that appear in the 2019 S&P Index and analyze their forecasted vs. actual price earnings for the last 20 years.

To ensure consistency in analysis among multiple firms, I divide both the forecasted and actual price earning dates by *calendar period* instead of fiscal period. This is because fiscal period differs by firm whereas calendar period is consistent by dates. 

#### Through the Bloomberg Excel functions, I gathered four datasets with different purposes:

- historical forecasted EPS
- historical actual EPS
- historical actual EOD price
- historical forecasted EPS relying on terms

---
Before delving into the data, let's define what above terms:

**EPS**

> EPS stands for ***Estimated Price Earnings.*** The formal definition of EPS given by Investopedia is this:

Earnings per share is the portion of a company's profit that is allocated to each outstanding share of a common stock, serving as an indicator of the company's financial health.

In other words, the EPS is a portion of the company's **net income** after all of their dividends are paid off. Dividends are profits that are paid out to shareholders of the company. EPS is one of the most useful and valuable financial measurements because they ***determine a stock's worth.*** The higher the stock, the more the company can pay out dividends to its shareholders, and the more net profit they are determined to generate.

$$ EPS = \frac{Net Income - Preferred Dividends}{Weighted Average Common Shares Outstanding}\$$

**EOD**

> EOD stands for the ***End of Day*** price. For any given day, the EOD marks the ***price at which the stock was valued*** at the end of the day's trading period.

In [3]:
PATH = './data/'
PATH_CLEAN = './data/clean/'
PATH_CLEAN_AVGS = './data/clean/averages/'
PATH_CLEAN_AVGS_SUB = './data/clean/averages/components/'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from functools import reduce

### Import Data <a id = "import"></a>

### Let's summarize the contents of the following DataFrames as we import them.
> All DataFrames consist of the 505 firms found in the 2019 S&P Index with EPS and EOD data encompassing 20 years: from January 1999 until the December 2019.

**Historic forecasted EPS**
> According to Investopedia, consensus estimates is normally an average or median of all the forecasts from individual analysts tracking a particular stock. In this case, the consensus estimate is for ***EPS for each firm present in the index as of 2019.*** Forecasted EPS is calculated by ***quarterly earnings,*** usually by each firm's fiscal period. Estimates of quarterly earnings are published at the beginning of each quarterly period.

In [4]:
#historic forecasted EPS 
eps_fc = pd.read_csv(PATH + 'sp-eps-fc.csv')

In [5]:
eps_fc.head()

,Term Forecasted,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,1999Q1,NaN,NaN,NaN,0.025,NaN,NaN,-0.10,0.423,NaN,...,0.456,0.090,0.239,0.123,NaN,NaN,0.123,NaN,0.622,NaN
1,1999Q2,NaN,NaN,NaN,0.020,NaN,NaN,-0.13,0.420,NaN,...,0.213,0.090,0.264,0.135,NaN,NaN,0.150,NaN,0.661,NaN
2,1999Q3,NaN,NaN,NaN,0.023,NaN,NaN,-0.11,0.381,NaN,...,0.767,0.099,0.300,0.129,NaN,NaN,0.171,NaN,0.693,NaN
3,1999Q4,NaN,NaN,NaN,0.016,NaN,NaN,NaN,0.432,NaN,...,0.432,0.124,0.386,0.178,NaN,NaN,0.164,NaN,0.667,NaN
4,00Q1,NaN,NaN,NaN,0.032,NaN,NaN,NaN,0.441,NaN,...,0.283,0.143,0.436,0.138,NaN,NaN,0.138,NaN,0.668,NaN


**Historic actual EPS**
> Unlike forecasted EPS, actual EPS are the real numbers denoting Earnings-per-Share for a singular firm. Historic actual EPS will be compared to forecasted EPS to draw correlations and comparisons.

In [6]:
#historic actual EPS
eps_act = pd.read_csv(PATH + 'sp-eps-act.csv')

In [7]:
eps_act.head()

,Quarter,Year,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,Q1,1999,0.16,0.99,NaN,0.040000,NaN,0.0925,-0.030,0.44,...,0.34,0.0000,0.210,0.123333,1.64,NaN,0.1725,NaN,0.56,NaN
1,Q2,1999,0.35,1.76,NaN,0.035357,NaN,0.1000,-0.145,0.42,...,0.06,0.0975,0.285,0.133333,2.28,NaN,0.2900,NaN,0.60,NaN
2,Q3,1999,0.30,1.86,NaN,0.050357,NaN,0.1075,-0.135,0.30,...,0.63,0.0325,0.315,0.130000,1.96,NaN,0.3200,NaN,0.64,NaN
3,Q4,1999,0.32,1.89,NaN,0.024643,NaN,0.0375,-0.080,0.43,...,0.43,0.1300,0.600,0.180000,1.72,NaN,0.2375,NaN,0.49,NaN
4,Q1,2000,0.29,0.89,NaN,0.040585,NaN,0.1050,-0.045,0.45,...,0.45,0.1650,0.500,0.140000,-1.48,NaN,0.2025,NaN,-0.33,NaN


**Historic actual EOD**
> Though this is not directly related to EPS data, EOD would be an interesting measure to use when generating intriguing visualizations and analyses. Who knows what visuals and conclusions I would arrive to with this measure. 

In [8]:
#historic actual EOD
eod_act = pd.read_csv(PATH + 'sp-eod-act.csv')

In [9]:
eod_act.head()

,date,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,3/31/1999,NaN,NaN,NaN,1.2835,NaN,8.2934,6.250,20.9504,NaN,...,NaN,20.2813,35.2813,7.7500,140.6214,NaN,12.6284,NaN,66.5000,NaN
1,6/30/1999,NaN,NaN,NaN,1.6540,NaN,6.1859,6.875,20.3630,NaN,...,NaN,28.6250,38.5625,9.6250,155.6057,NaN,9.7297,NaN,63.5000,NaN
2,9/30/1999,NaN,NaN,NaN,2.2612,NaN,5.7462,7.750,16.4471,NaN,...,NaN,32.7656,37.9688,7.5833,110.4883,NaN,7.3591,NaN,55.1250,NaN
3,12/31/1999,52.0909,NaN,NaN,3.6719,NaN,3.6843,18.375,16.2513,NaN,...,NaN,45.4688,40.2813,7.8750,59.7723,NaN,6.9434,NaN,59.1875,NaN
4,3/31/2000,70.0721,NaN,NaN,4.8504,NaN,3.6388,20.250,15.7478,NaN,...,NaN,82.8125,38.9063,9.4583,68.4994,NaN,5.5839,NaN,41.6250,NaN


**Historic forecasted EPS 3 months prior**
> Instead of using forecast data collected at the beginning of the fiscal period, this feature contains EPS data projected 3 months before the current fiscal period. This is an interesting metric to see how differently forecasters make their predictions at different times.

In [10]:
#historic forecasted EPS 3-months prior
eps_fc_terms = pd.read_csv(PATH + 'sp-eps-fc-terms.csv')

In [11]:
eps_fc_terms.head()

,Forecast Made,Term Forecasted,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,10/1/1999,00Q1,NaN,NaN,NaN,0.030,NaN,NaN,NaN,0.466,...,NaN,0.143,0.398,0.140,NaN,NaN,0.144,NaN,0.777,NaN
1,1/1/2000,00Q2,NaN,NaN,NaN,0.026,NaN,NaN,NaN,0.437,...,NaN,0.168,0.378,0.153,NaN,NaN,0.184,NaN,0.790,NaN
2,4/1/2000,00Q3,NaN,NaN,NaN,0.029,NaN,NaN,NaN,0.414,...,0.84,0.189,0.425,0.149,NaN,NaN,0.218,NaN,0.803,NaN
3,7/1/2000,00Q4,NaN,NaN,NaN,0.032,NaN,NaN,-0.18,0.477,...,0.53,0.211,0.487,0.200,NaN,NaN,0.244,NaN,0.777,NaN
4,9/1/2000,01Q1,NaN,NaN,NaN,0.041,NaN,NaN,-0.09,0.495,...,0.40,0.259,0.475,0.158,NaN,NaN,0.178,NaN,0.750,NaN


### B) Assess <a id = "assess"></a>

> The following DataFrames contain data for each firm across various dates. To account for all firm averages, my goal is to generate a CSV file where each row contains the firm average, with the features as columns.

In [12]:
dict_dfs = {'eps_fcast' : eps_fc,
           'eps_actual' : eps_act,
           'eod_actual' : eod_act,
           'eps_fcast_terms' : eps_fc_terms}

### Check Missing Data

In [13]:
for key, df in dict_dfs.items():
    print(key, df.shape)

eps_fcast (84, 506)
eps_actual (84, 507)
eod_actual (84, 506)
eps_fcast_terms (80, 507)


***We need to make sure the number of firms in each DataFrame is consistent.***

---

**Observation 1:** for `eps_fcast`, there are 505 firms encompassing 84 quarterly fiscal periods since 1999.

> There are 506 columns: 1 column being `Term Forecast`, the rest firm names.

**Observation 2:** for `eps_actual`, there are 505 firms encompassing 84 quarterly fiscal periods since 1999.

> There are 507 columns: 2 columns being `Quarter` and `Year`, the rest firm names.

**Observation 3:** for `eod_actual`, there are 505 firms encompassing 84 quarterly calendar periods since 1999.

> There are 506 columns: 1 column being `date`, the rest firm names.

**Observation 4:** for `eps_fcast_terms`, there are 505 firms encompassing 80 quarterly calendar periods since 1999.

> There are 507 columns: 2 columns being `Forecast Made` and `Term Forecasted`, the rest firm names.
 
Since there are only 80 quarterly calendar periods, that ***implies an entire year is missing.***

**Observation 5:** For `eps_fcast`, `eps_actual`, and `eod_actual`, since there are 4 quarters in a year, 84 quarterly forecast periods equate to 21 years. This is correct since we are analyzing the years from 1999 until the end of 2019.

### Most importantly, the number of firms across all DataFrames is consistent.

----

***Check which year is missing from `eps_fcast_terms`***.

> Each **Term Forecasted** entry under `eps_fcast_terms` records the year with 2 digits, so Quarter 1 of the year 2000 becomes 00Q1.
- isolate the first 2 characters to get the year
- join a '20' in front of the string so 00 becomes 2000
- list the number of unique values.

In [14]:
#iterate over all years in eps_fcast_terms, append '20' in front of string
list(map(lambda x: '20' + x, eps_fc_terms['Term Forecasted'].str[:2].unique()))

['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019']

**Observation**: The year 1999 is missing from `eps_fc_terms`. This makes sense because the start of the forecasting period would be in the last quarter of 1999, which is October. 

---

### Check Nulls

In [15]:
for key, df in dict_dfs.items():
    #display (row, column) per DataFrame
    print('# NaN in {}: {}'.format(key, df.isna().sum().sum()))

# NaN in eps_fcast: 7055
# NaN in eps_actual: 5021
# NaN in eod_actual: 6921
# NaN in eps_fcast_terms: 7080


**Observation:** All four DataSets contain null values.
> In order to combat this, we'll have to look at both the **number of rows** and **number of columns** with missing data, separately. This way, we can isolate which firms and/or time periods contain complete or incomplete data.

In [16]:
#check rows for missing data
for key, df in dict_dfs.items():
    num_rows_missing = df.isna().sum().max()
    print('{} has {} time periods containing missing data, out of {} total rows.'.format(key, num_rows_missing, df.shape[0]))

eps_fcast has 82 time periods containing missing data, out of 84 total rows.
eps_actual has 84 time periods containing missing data, out of 84 total rows.
eod_actual has 83 time periods containing missing data, out of 84 total rows.
eps_fcast_terms has 80 time periods containing missing data, out of 80 total rows.


In [17]:
len(eps_fc.columns[eps_fc.isnull().any()])

254

In [18]:
#check columns for missing data
for key, df in dict_dfs.items():
    cols_missing = df.columns[df.isnull().any()]
    num_cols_missing = len(cols_missing)
    print('{} has {} firms containing missing data, out of {} total columns.'
         .format(key, num_cols_missing, df.shape[1]))

eps_fcast has 254 firms containing missing data, out of 506 total columns.
eps_actual has 432 firms containing missing data, out of 507 total columns.
eod_actual has 505 firms containing missing data, out of 506 total columns.
eps_fcast_terms has 340 firms containing missing data, out of 507 total columns.


**Observation 1:** The only two datasets with incomplete data for all time periods are **actual EPS** and **forecasted EPS 3 months prior.**

> To address this problem, it'd be helpful to isolate the time period ranges for the datasets with incomplete data by row, **forecasted EPS** and **actual EOD price.**

**Observation 2:** For all datasets, all firms contain incomplete data across all time periods.
> This is expected, as analyzing financial history spanning over 20 years will naturally be rife with missing and inaccurate data. The ***good news is that `eps_fcast`, `eps_actual`, and `eps_fcast_terms` are the most complete, while `eod_actual` contains the most amount of missing data.***


**Moving forward, we need to make sure that these inconsistencies won't clash with our analysis.**
> **My approach:** instead of looking at rows and columns ***with*** missing data, we'll be looking at rows and columns that ***are all missing data.***

I figured that if there is some missing data here and there scattered throughout the matrix, then that should not skew our analysis too much.

However, if there a significant amount of rows/columns that are entirely empty, then we ***might have to get ready to drop some dates and firms from our data overall.***

In [19]:
#check for empty rows, return False if row contains at least one non-null value, True if all are null
for key, df in dict_dfs.items():
    cols_check = df.columns
    num_empty_rows = (df[cols_check].isnull().apply(lambda x: all(x), axis = 1)).value_counts()
    print(key, '\n', num_empty_rows, '\n----')

eps_fcast 
 False    84
dtype: int64 
----
eps_actual 
 False    84
dtype: int64 
----
eod_actual 
 False    84
dtype: int64 
----
eps_fcast_terms 
 False    80
dtype: int64 
----


**Observation:** All datasets do not contain empty rows.
> This is good news, since we can rely on the firms' averages per row instead of having to drop or limit time periods.

In [20]:
#check for empty columns, return False if column contains at least one non-null value, True if all are null
for key, df in dict_dfs.items():
    cols_check = df.columns
    num_empty_cols = df[cols_check].isnull().apply(lambda x: all(x), axis = 0).value_counts()
    print(key, '\n', num_empty_cols)

eps_fcast 
 False    506
dtype: int64
eps_actual 
 False    506
True       1
dtype: int64
eod_actual 
 False    506
dtype: int64
eps_fcast_terms 
 False    506
True       1
dtype: int64


**Observation:** `eps_actual` and `eps_fcast_terms` are the only datasets that have an empty column.
> Let's isolate and look at the singular empty column for both DataFrames.

In [21]:
#create function to return an array of column names containing empty data
def comb_cols(df):
    empty_cols = []
    for column in df:
        if df[column].isnull().all():
            empty_cols.append(column)
            
    return empty_cols

In [22]:
#comb datasets for empty columns
print('In eps_act, the firm {} has no data.'.format(comb_cols(eps_act)))
print('In eps_fc_terms, the firm {} has no data.'.format(comb_cols(eps_fc_terms)))

In eps_act, the firm ['AMCR UN Equity'] has no data.
In eps_fc_terms, the firm ['AMCR UN Equity'] has no data.


**Observation:** The same firm in both datasets is empty of data.
> Though this is an annoying error to deal with, it still is to our advantage that both datasets ***share one firm*** in common for missing data. This way, we don't have to worry about dropping two entire firms.

---

### Check Duplicate Data

In [23]:
#check for duplicate data across all rows and columns
for key, df in dict_dfs.items():
    print(key, df.duplicated().sum())

eps_fcast 0
eps_actual 0
eod_actual 0
eps_fcast_terms 0


**Observation:** For all datasets, there are ***no duplicate data.*** This is good news!

**Next, I will check for duplicated firm names.** Although the presence of duplicated firm names will inherently imply duplicated data, sometimes data gets dispersed in weird, unexpected ways, especially when dealing with large datasets.

In [24]:
#check for duplicated firm names
for key, df in dict_dfs.items():
    print(key, df.columns.duplicated().sum())

eps_fcast 0
eps_actual 0
eod_actual 0
eps_fcast_terms 0


**Observation:** For all datasets, there are ***no duplicate firm names.*** This is also good news.

### There is no need to dedupe our data during the cleaning stage.

---

### Check Data Types

> Make sure all numerical data types are consistent.
- EPS and EOD values must all be **floats**
- Dates should all be **objects** (for now)

- `eps_fc` contains ***1 date field,***, so we should expect ***1 object type.***
- `eps_act` contains ***2 date fields,*** so we should expect ***2 object types.***
- `eod_act` contains ***1 date field,*** so we should expect ***1 object type.***
- `eps_fc_terms` contains ***2 date fields,*** so we should expect ***2 object types.***

> And everything else should be a ***float*** type.

**Examine each dataset's overall data types per column.**

In [25]:
for key, df in dict_dfs.items():
    print(key,'\n', df.dtypes.value_counts(), '\n--------')

eps_fcast 
 float64    505
object       1
dtype: int64 
--------
eps_actual 
 float64    499
object       7
int64        1
dtype: int64 
--------
eod_actual 
 float64    505
object       1
dtype: int64 
--------
eps_fcast_terms 
 float64    505
object       2
dtype: int64 
--------


**Observation 1:** `eps_fc`, `eod_act`, and `eps_fc_terms` are all consistent with the expected number of object types.

> But we'll still have to double check these facts later.

**Observation 2:** `eps_act` contains 5 more object columns than expected.

---

***Isolate 'Object' Columns under `eps_act`***

In [26]:
eps_act.select_dtypes(include = 'object').head()

,Quarter,BRK/B UN Equity,FOX UW Equity,GOOG UW Equity,HCP UN Equity,SYMC UW Equity,UA UN Equity
0,Q1,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
1,Q2,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
2,Q3,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
3,Q4,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
4,Q1,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable


**Observation:** These firms are all 'Object' types because there is no recorded data under them.
> This is actually missing data. From `eps_act`, there are 6 firms that are empty of data.

### This was a tricky situation to spot, but I caught it in the end. There actually is missing data after all, but we missed those 5 entire firms during the "Check Missing Data" stage, because they were recorded as objects. This means that Pandas wrongly recognized these 5 firms as "complete."

### Quality

**Missing Data**
> **In the end, I decided not to drop any rows or columns with missing data. For the sake of data preservation, I kept all missing data recorded as NaN.**

-  `eps_fc_terms` is missing the year 1999.
- `eps_act` and `eps_fc_act` have one firm with empty data: 'AMCR UN Equity'
- `eps_act` contains 7 empty firms: BRK/B UN Equity, FOX UW Equity, GOOG UW Equity, HCP UN Equity, SYMC UW Equity, UA UN Equity
--- 
- Unnormalized date formats among all DataFrames.
- Firm names across all DataFrames are capitalized and contain white space.
- Erroneous data type for `eps_act` Object columns.
- Erroneous data type for **date** under `eod_act`
- Erroneous data type for **forecast_made** under `eps_fc_terms`
- Erroneous data types for **term** under `eps_act`, `eps_fc`, and `eps_fc_terms` to DateTime quarter index
- No recorded quarterly data for `eod_act`
---
- Firm names not referenced by **firm_id**
- No recorded 20-year for each dataset.
- No recorded yearly averages for each dataset
- No recorded quarterly averages for each dataset.
---
- Features datasets 

### Tidiness
- Unnormalized data among `df_twenty_year_avgs`, `df_yearly_avgs`, and `df_quarter_avgs`
- 20-year, yearly, and quarterly averages contained in different DataFrames.
- Unnormalized data among `eps_act`, `eps_fc`, `eod_act`, and `eps_fc_terms`


### C) CLEAN <a id="clean"></a>

In [619]:
eps_fc_clean = eps_fc.copy()
eps_act_clean = eps_act.copy()
eod_act_clean = eod_act.copy()
eps_fc_terms_clean = eps_fc_terms.copy()

In [620]:
#create dictionary of all clean dfs for iteration
dict_clean = {'eps_fc_clean' : eps_fc_clean,
             'eps_act_clean' : eps_act_clean,
             'eod_act_clean' : eod_act_clean,
             'eps_fc_terms_clean' : eps_fc_terms_clean}

### Code
> **ISSUE 1:**  Unnormalized date formats among all DataFrames.

**Define:** 
- Conjoin `eps_act_clean` dates from 2 columns into 1 to match `eps_fc_clean` format.
> The format we want is YYYYQN. For example, Quarter 1 in 2005 will be 2005Q1.

- Rename both fiscal periods under `eps_act_clean` and `eps_fc_clean` to **term.**

- For both fiscal periods under `eps_fc_terms_clean`, add an underscore.
- Under `eps_fc_terms_clean`, format **term** to YYYYQN.

In [621]:
#create new column, combine year and quarter into 1 string
eps_act_clean['term'] = eps_act_clean['Year'].map(str) + eps_act_clean['Quarter']

In [622]:
#drop 'Year' and 'Quarter'
eps_act_clean.drop(['Year', 'Quarter'], axis = 1, inplace = True)

In [623]:
#designate new column as first column
reorder_cols = eps_act_clean.columns.tolist()
reorder_cols.insert(0, reorder_cols.pop(reorder_cols.index('term')))

eps_act_clean = eps_act_clean.reindex(columns = reorder_cols)

In [624]:
#rename eps_fc_clean fiscal period to term
eps_fc_clean.rename(index = str, columns = {'Term Forecasted' : 'term'}, inplace = True)

#reassign to dictionary
dict_clean['eps_act_clean'] = eps_act_clean

In [625]:
eps_fc_clean[4:].term = '20' + eps_fc_clean[4:].term.astype(str)

D:\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [626]:
#add an underscore for both date periods under eps_fc_terms_clean
eps_fc_terms_clean.rename(index = str, columns = {'Forecast Made' : 'forecast_made',
                                                 'Term Forecasted' : 'term'},
                         inplace = True)

In [627]:
#under term, add a '20' before each string
eps_fc_terms_clean.term = '20' + eps_fc_terms_clean.term.astype(str)

### Test

In [628]:
#give overview of both DataFrames
eps_fc_clean.sample(3)

,term,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
63,2014Q4,0.888,1.512,1.471,1.303,0.856,1.052,0.063,0.686,1.101,...,0.344,0.555,1.339,0.608,NaN,0.599,0.662,1.705,NaN,0.357
27,2005Q4,0.363,-2.360,0.361,0.053,NaN,0.239,0.000,0.758,0.359,...,0.256,0.202,1.451,0.350,1.28,NaN,0.390,0.827,1.348,NaN
83,2019Q4,0.855,1.269,1.365,2.837,2.200,1.583,1.084,0.947,1.713,...,0.528,1.034,0.747,0.749,1.13,0.900,1.138,2.265,1.102,0.880


In [629]:
eps_act_clean.sample(3)

,term,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
32,2007Q1,0.37,0.35,0.72,0.168571,NaN,0.320000,-0.23,0.45,0.474818,...,0.29,0.24,1.64,0.380000,1.00,NaN,0.365,0.99,1.38,NaN
15,2002Q4,-0.51,-3.39,NaN,-0.009286,NaN,0.222500,-0.32,0.40,0.080000,...,-0.52,0.10,0.60,0.270000,0.04,NaN,0.290,0.37,0.96,NaN
75,2017Q4,0.55,-1.22,2.50,2.080000,0.03,-1.577115,0.34,-0.48,1.510000,...,0.37,0.62,1.97,-2.845144,-0.76,0.395323,1.290,6.08,0.57,0.17


In [630]:
#check consistent format between both `term` fields
forecasts = pd.concat([eps_fc_clean.term, eps_act_clean.term])

#count number of instances where unique term values are the same
(forecasts.value_counts().sort_index() == 2).value_counts()

True    84
Name: term, dtype: int64

In [631]:
#check that dates are the first columns
for key, df in dict_clean.items():
    print(df.columns[:2])

Index(['term', 'A UN Equity'], dtype='object')
Index(['term', 'A UN Equity'], dtype='object')
Index(['date', 'A UN Equity'], dtype='object')
Index(['forecast_made', 'term'], dtype='object')


## The **term** column under `eps_fc_clean`, `eps_act_clean`, and `eps_fc_terms_clean` are all normalized to YYYQN format. This means `eod_act_clean` is the only one with a different date formatting rule.

### Code
> **ISSUE 2:** Firm names across all DataFrames are capitalized and contain white space.

**Define:** Iterate across all DataFrames. Get rid of everything from the first whitespace character onward. Lowercase column names.

In [632]:
#lowercase names
for key, df in dict_clean.items():
    lower_cols = [x.lower() for x in df.columns]
    df.columns = lower_cols

In [633]:
#get rid of white space
for key, df in dict_clean.items():
    stock_symbols = [x.split(' ', 1)[0] for x in df.columns]
    df.columns = stock_symbols

### Test

In [634]:
#check that all firm names are consistent, should return 505 firm results
all_ticks = []
for key, df in dict_clean.items():
    firm_ticks = np.array(df.columns.values)
    all_ticks.append(firm_ticks)
    
#flatten array
all_ticks = np.hstack(all_ticks)

#assign tick names to Pandas Series
all_ticks = pd.Series(all_ticks)

In [635]:
#look at lowest 3 value counts, SHOULD BE DATE PERIODS
all_ticks.value_counts().sort_values()[:5]

forecast_made    1
date             1
term             3
acn              4
wat              4
dtype: int64

In [636]:
#look at highest value counts, should be FOUR (because there are 4 dataframes all with firm names)
all_ticks.value_counts().sort_values(ascending = False)[:5]

khc     4
wmt     4
pcar    4
mlm     4
lkq     4
dtype: int64

In [637]:
#check for any upper case instance among columns
all_ticks.str.isupper().any()

False

In [638]:
#check for any white space
all_ticks.str.contains(' ').any()

False

### Code
> **ISSUE 3:** Erroneous data type for `eps_act` Object columns.

**Define:**

- Convert '#N/A Field Not Applicable' strings into 'NaN'
- Convert the 6 `eps_act_clean` Object columns into NaN type.

In [639]:
#isolate 6 firm names recorded as 'object'
obj_ticks = eps_act_clean.select_dtypes(include = 'object').columns.values

#exclude 'term'
obj_ticks = obj_ticks[1:]

In [640]:
#convert to NaN
str_replace = '#N/A Field Not Applicable'
eps_act_clean[obj_ticks] = eps_act_clean[obj_ticks].replace(str_replace, np.nan)

### Test

In [641]:
eps_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 506 entries, term to zts
dtypes: float64(505), object(1)
memory usage: 332.1+ KB


In [642]:
#make sure 'term' is the only object-type column
eps_act_clean.select_dtypes(include = 'object').columns.values.tolist()

['term']

### Code

> **ISSUE 4:**  Erroneous data type for **date** under `eod_act`

**Define:** Convert column `date` to DateTime object.

In [643]:
eod_act_clean.date = pd.to_datetime(eod_act_clean.date, infer_datetime_format=True)

### Test

In [644]:
eod_act_clean.sample(2)

,date,a,aal,aap,aapl,abbv,abc,abmd,abt,acn,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
57,2013-06-28,30.5771,NaN,81.17,56.5829,41.34,55.8300,21.56,34.8800,71.96,...,NaN,39.61,90.350,40.9600,23.8958,26.94,49.8592,74.94,28.88,30.89
9,2001-06-29,21.8975,NaN,NaN,1.6607,NaN,13.4149,23.58,21.4863,NaN,...,NaN,41.24,43.675,14.7833,25.2131,NaN,7.8916,NaN,59.00,NaN


In [645]:
#confirm datetime64 present
eod_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 506 entries, date to zts
dtypes: datetime64[ns](1), float64(505)
memory usage: 332.1 KB


### Code
> **ISSUE 5:** Erroneous data type for **forecast_made** under `eps_fc_terms`.

**Define:**

- Convert column **forecast_made** to DateTime object.

In [646]:
eps_fc_terms_clean.forecast_made = pd.to_datetime(eps_fc_terms_clean.forecast_made, infer_datetime_format=True)

### Test

In [647]:
#confirm presence of datetime64 object
eps_fc_terms_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 0 to 79
Columns: 507 entries, forecast_made to zts
dtypes: datetime64[ns](1), float64(505), object(1)
memory usage: 317.5+ KB


In [648]:
eps_fc_terms_clean.sample(2)

,forecast_made,term,a,aal,aap,aapl,abbv,abc,abmd,abt,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
10,2002-04-01,2002Q3,NaN,NaN,0.276,0.010,NaN,NaN,-0.323,0.545,...,0.957,0.042,0.449,0.220,NaN,NaN,0.227,0.273,0.966,NaN
46,2011-04-01,2011Q3,0.659,0.102,1.207,0.758,NaN,0.586,-0.090,1.158,...,0.644,0.516,1.961,0.473,1.06,NaN,0.840,1.064,0.095,NaN


### Code
> **ISSUE 6:** Erroneous data types for **term** under `eps_act`, `eps_fc`, and `eps_fc_terms` to DateTime quarter index

**Define:** Convert YYYYQQ formats into DateTime quarter index type

In [649]:
#helper function to turn a column into Quarter Period type
def convert_period(df, col):
    return pd.to_datetime(df[col]).dt.to_period('Q')
# eps_fc_terms_clean.term = pd.PeriodIndex(eps_fc_terms_clean.term, freq='Q')

In [650]:
#convert term fields to Period type
eps_fc_clean.term = convert_period(eps_fc_clean, 'term')
eps_act_clean.term = convert_period(eps_act_clean, 'term')
eps_fc_terms_clean.term = convert_period(eps_fc_terms_clean, 'term')

### Test

In [651]:
#verify Period Object types
print(eps_fc_clean.dtypes['term'])
print(eps_act_clean.dtypes['term'])
print(eps_fc_terms_clean.dtypes['term'])

period[Q-DEC]
period[Q-DEC]
period[Q-DEC]


### Code
> **ISSUE 7:** No recorded quarterly data for `eod_act`

**Define:** Add a new column **term**

In [652]:
eod_act_clean['term'] = eod_act_clean.date.dt.to_period('Q')

In [653]:
#reorder columns
eod_cols = ['date', 'term'] + [col for col in eod_act_clean.iloc[:, 1:] if col!='term']

In [654]:
eod_act_clean = eod_act_clean.reindex(columns = eod_cols)

### Test

In [655]:
#verify column order
eod_act_clean.head()

,date,term,a,aal,aap,aapl,abbv,abc,abmd,abt,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
0,1999-03-31,1999Q1,NaN,NaN,NaN,1.2835,NaN,8.2934,6.250,20.9504,...,NaN,20.2813,35.2813,7.7500,140.6214,NaN,12.6284,NaN,66.5000,NaN
1,1999-06-30,1999Q2,NaN,NaN,NaN,1.6540,NaN,6.1859,6.875,20.3630,...,NaN,28.6250,38.5625,9.6250,155.6057,NaN,9.7297,NaN,63.5000,NaN
2,1999-09-30,1999Q3,NaN,NaN,NaN,2.2612,NaN,5.7462,7.750,16.4471,...,NaN,32.7656,37.9688,7.5833,110.4883,NaN,7.3591,NaN,55.1250,NaN
3,1999-12-31,1999Q4,52.0909,NaN,NaN,3.6719,NaN,3.6843,18.375,16.2513,...,NaN,45.4688,40.2813,7.8750,59.7723,NaN,6.9434,NaN,59.1875,NaN
4,2000-03-31,2000Q1,70.0721,NaN,NaN,4.8504,NaN,3.6388,20.250,15.7478,...,NaN,82.8125,38.9063,9.4583,68.4994,NaN,5.5839,NaN,41.6250,NaN


In [656]:
#verify 84 columns
eod_act_clean.shape[0]

84

In [657]:
eod_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 507 entries, date to zts
dtypes: datetime64[ns](1), float64(505), period[Q-DEC](1)
memory usage: 332.8 KB


## All full years are normalized to YYYY-MM-DD, and all quarterly records to YYYY-QQ.

## This is to enable for more efficient handling, cleaning, and classifying of data later on.

### Code

**ISSUE 8:** Firm names not referenced by **firm_id**

**Define:**
- Assign a **firm_id** to each firm for future normalization.

- Generate new CSV named `firms.csv`

In [658]:
#get firm names
firm_names = eps_act_clean.columns[1:].to_numpy()

In [659]:
#assign ids to firm names
firm_names_ids = {}
counter = 0

for firm in firm_names:
    firm_names_ids.update({firm: counter})
    counter += 1

In [660]:
#generate new DF
firm_ids = pd.DataFrame(list(firm_names_ids.items())).rename(columns = {0: 'firm', 1: 'firm_id'})

In [661]:
#switch firm and firm_id columns
firm_ids = firm_ids.reindex(columns = ['firm_id', 'firm'])

### Test

In [662]:
firm_ids.sample(10)

,firm_id,firm
479,479,wba
238,238,hsy
52,52,atvi
223,223,hes
11,11,adm
180,180,fast
135,135,dfs
493,493,wynn
153,153,dvn
100,100,cinf


In [663]:
firm_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 2 columns):
firm_id    505 non-null int64
firm       505 non-null object
dtypes: int64(1), object(1)
memory usage: 8.0+ KB


### Code
> **ISSUE 9:** No recorded 20-year averages for each dataset.

**Define:**


- Isolate 20-year averages for each firm into its own DataFrame

- Create new DataFrame `twenty_avgs` depicting all 20-year averages for each firm

In [718]:
#convert foreign keys to str temporarily
firm_ids_dict = {str(k):v for k, v in firm_names_ids.items()}

In [724]:
def get_twenty_yr_avgs(df, index_names, col_name, firm_ids_dict):
    df = pd.DataFrame({col_name: df[index_names].mean()}).rename_axis('firm_id').reset_index()
    
    #set 'firm_id' type int64
    df.applymap(lambda s: firm_ids_dict.get(s) if s in firm_ids_dict else s)
    
    return df

In [725]:
#get 20-year averages
eps_fc_twenty_avg = get_twenty_yr_avg(eps_fc_clean, firm_names, 'eps_fc', firm_ids_dict)
eps_act_twenty_avg = get_twenty_yr_avg(eps_act_clean, firm_names, 'eps_act', firm_ids_dict)
eod_act_twenty_avg = get_twenty_yr_avg(eod_act_clean, firm_names, 'eod_act', firm_ids_dict)
eps_fc_terms_twenty_avg = get_twenty_yr_avg(eps_fc_terms_clean, firm_names, 'eps_fc_terms', firm_ids_dict)

In [726]:
#put averages into a list
twenty_avg_list = [eps_fc_twenty_avg,
                  eps_act_twenty_avg,
                  eod_act_twenty_avg,
                  eps_fc_terms_twenty_avg]

In [727]:
#merge all Series
df_twenty_year_avgs = reduce(lambda x, y: pd.merge(x, y, on = 'firm_id', how = 'outer'), twenty_avg_list)

### Test

In [681]:
df_twenty_year_avgs.sample(5)

,firm_id,eps_fc,eps_act,eod_act,eps_fc_terms
503,503,0.961549,0.445301,43.716448,0.716300
69,69,9.167367,5.120181,597.128590,10.053804
10,10,0.649864,0.478036,68.088167,0.672000
166,166,0.204017,0.655964,46.418647,0.238685
5,5,0.824586,0.475927,40.691608,0.848982


In [728]:
df_twenty_year_avgs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505 entries, 0 to 504
Data columns (total 5 columns):
firm_id         505 non-null int64
eps_fc          505 non-null float64
eps_act         498 non-null float64
eod_act         505 non-null float64
eps_fc_terms    504 non-null float64
dtypes: float64(4), int64(1)
memory usage: 23.7 KB


In [683]:
#check there are 505 firms
df_twenty_year_avgs.firm_id.nunique()

505

In [684]:
#check missing data
df_twenty_year_avgs.isna().any()

firm_id         False
eps_fc          False
eps_act          True
eod_act         False
eps_fc_terms     True
dtype: bool

In [685]:
#check duplicate data
df_twenty_year_avgs.duplicated().any()

False

### Code
> **ISSUE 10:** No recorded yearly averages for each dataset.

**Define:**
- Create 4 separate DataFrames for all attributes
- Rename columns to "Feature_Year" (e.g. eps_fc_1999, eod_act_2000, etc.)
- Outer merge all DataFrames to create new DataFrame, `df_yearly_avgs`, on **firms** attribute

In [744]:
#helper function to calculate yearly average of each DataFrame
def calculate_df_yearly_avgs(df, col, firm_ids_dict):
    #extract year from dates, reset index, and remove index name
    df = df.groupby(df[col].dt.year).mean().transpose().rename_axis('firm_id').reset_index()
    
    df['firm_id'] = df['firm_id'].map(lambda s: firm_ids_dict.get(s) if s in firm_ids_dict else s)
    return df

In [745]:
#helper function to melt columns
def melt_avgs(df, firm_id, var_name):
    return df.melt(id_vars = [firm_id],
                value_vars = df.columns.difference([firm_id]),
                var_name = var_name,
                value_name = 'average')

In [746]:
#calculate and melt
def create_df_yearly_avgs(df, col, firm_ids_dict, firm_id):
    df = calculate_df_yearly_avgs(df, 'term', firm_ids_dict)
    df = melt_avgs(df, firm_id, 'year')
    return df

In [747]:
#generate DataFrames of all yearly averages
df_eps_fc_yearly_avgs = create_df_yearly_avgs(eps_fc_clean, 'term', firm_ids_dict, 'firm_id')
df_eps_act_yearly_avgs = create_df_yearly_avgs(eps_act_clean, 'term', firm_ids_dict, 'firm_id')

#for eod_act, doesn't matter if we use date or term field
df_eod_act_yearly_avgs = create_df_yearly_avgs(eod_act_clean, 'date', firm_ids_dict, 'firm_id')
df_eps_fc_terms_yearly_avgs = create_df_yearly_avgs(eps_fc_terms_clean, 'term', firm_ids_dict, 'firm_id')

In [749]:
def store_df(path, df, csv_name):
    df.to_csv(path + csv_name, encoding = 'utf-8', index = False)

In [750]:
#create copies
eps_fc_yearly_avgs = df_eps_fc_yearly_avgs.copy()
eps_act_yearly_avgs = df_eps_act_yearly_avgs.copy()
eod_act_yearly_avgs = df_eod_act_yearly_avgs.copy()
eps_fc_terms_yearly_avgs = df_eps_fc_terms_yearly_avgs.copy()

In [751]:
#add new column entailing "type" of feature before concat
eps_fc_yearly_avgs = eps_fc_yearly_avgs.assign(feature = 'eps_fc')
eps_act_yearly_avgs = eps_act_yearly_avgs.assign(feature = 'eps_act')
eod_act_yearly_avgs = eod_act_yearly_avgs.assign(feature = 'eod_act')
eps_fc_terms_yearly_avgs = eps_fc_terms_yearly_avgs.assign(feature = 'eps_fc_terms')

In [752]:
yearly_avgs_arr = [eps_fc_yearly_avgs, 
                  eps_act_yearly_avgs,
                  eod_act_yearly_avgs,
                  eps_fc_terms_yearly_avgs]

In [753]:
#merge all yearly average DataFrames
df_yearly_avgs = pd.concat(yearly_avgs_arr)

### Test

In [755]:
#each average, individually

#eps_fc_yearly
eps_fc_yearly_avgs.sample(10)

,firm_id,year,average,feature
8396,316,2015,0.62950,eps_fc
9352,262,2017,0.73750,eps_fc
1701,186,2002,-0.02900,eps_fc
2469,449,2003,NaN,eps_fc
6504,444,2011,0.73975,eps_fc
6683,118,2012,0.13100,eps_fc
4108,68,2007,0.62375,eps_fc
10420,320,2019,0.49725,eps_fc
8487,407,2015,0.09375,eps_fc
7567,497,2013,1.90225,eps_fc


In [756]:
#eps_fc_yearly
eps_act_yearly_avgs.sample(10)

,firm_id,year,average,feature
1861,346,2002,-0.065000,eps_act
3621,86,2006,0.542500,eps_act
8599,14,2016,-0.360000,eps_act
157,157,1999,0.167500,eps_act
5248,198,2009,0.366250,eps_act
8864,279,2016,0.877500,eps_act
650,145,2000,0.094167,eps_act
9763,168,2018,1.477500,eps_act
7184,114,2013,1.855000,eps_act
4727,182,2008,NaN,eps_act


In [757]:
#eod_act_yearly
eod_act_yearly_avgs.sample(10)

,firm_id,year,average,feature
3980,445,2006,46.73750,eod_act
6108,48,2011,18.04160,eod_act
997,492,2000,19.55520,eod_act
5228,178,2009,6.47750,eod_act
10275,175,2019,75.35000,eod_act
6773,208,2012,NaN,eod_act
2876,351,2004,7.08915,eod_act
6989,424,2012,28.29000,eod_act
7450,380,2013,79.63375,eod_act
4295,255,2007,25.64230,eod_act


In [758]:
#eps_fc_terms_yearly
eps_fc_terms_yearly_avgs.sample(10)

,firm_id,year,average,feature
1257,247,2002,NaN,eps_fc_terms
181,181,2000,NaN,eps_fc_terms
959,454,2001,NaN,eps_fc_terms
578,73,2001,0.63500,eps_fc_terms
7091,21,2014,0.07825,eps_fc_terms
9166,76,2018,0.34375,eps_fc_terms
6791,226,2013,0.98325,eps_fc_terms
3438,408,2006,0.08675,eps_fc_terms
8133,53,2016,1.25075,eps_fc_terms
9706,111,2019,2.78100,eps_fc_terms


In [759]:
#verify all columns are consistent
print(eps_fc_yearly_avgs.columns, '\n',
      eps_act_yearly_avgs.columns, '\n',
      eod_act_yearly_avgs.columns, '\n',
      eps_fc_terms_yearly_avgs.columns)

Index(['firm_id', 'year', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'year', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'year', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'year', 'average', 'feature'], dtype='object')


In [760]:
#verify there are 505 firms in each dataframe
print(eps_fc_yearly_avgs.firm_id.nunique(), '\n',
      eps_act_yearly_avgs.firm_id.nunique(), '\n',
      eod_act_yearly_avgs.firm_id.nunique(), '\n',
      eps_fc_terms_yearly_avgs.firm_id.nunique())

505 
 505 
 505 
 505


In [761]:
#all yearly averages
df_yearly_avgs.sample(10)

,firm_id,year,average,feature
8414,334,2015,55.38500,eod_act
4891,346,2008,0.54225,eps_fc
5362,312,2009,0.71075,eps_fc
9711,116,2019,0.14700,eps_fc_terms
5469,419,2009,54.86200,eod_act
1084,74,2002,NaN,eps_fc_terms
6311,251,2011,0.28500,eps_act
246,246,1999,-0.11750,eps_act
2850,325,2004,NaN,eps_fc
7451,381,2013,28.31135,eod_act


In [762]:
#verify 505 firms in all yearly averages df
df_yearly_avgs.firm_id.nunique()

505

In [763]:
df_yearly_avgs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41915 entries, 0 to 10099
Data columns (total 4 columns):
firm_id    41915 non-null int64
year       41915 non-null object
average    35971 non-null float64
feature    41915 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 1.6+ MB


### Code

> **ISSUE 11:** No recorded quarterly averages for each dataset.

**Define:** 

- Parse `eod_act_clean` dates  by **calendar quarter average** with quarterly data in a new DataFrame.
- Create separate DataFrames containing quarterly averages for `eps_fc_clean`, `eps_act_clean`, and `eps_fc_terms_clean` 
- Rename columns to "Quarter_Year_Feature" (e.g. **q1_eps_fc,** etc.)
- Outer merge all DataFrames into a new df `df_quarterly_avgs` on **firms**

In [765]:
#helper function to calculate quarterly average of each DataFrame
def calculate_df_quarterly_avgs(df, col, firm_ids_dict):
    #extract quarter from Period objects
    df =  df.groupby(df[col].dt.strftime('Q%q')).mean().transpose().rename_axis('firm_id').reset_index()
    df['firm_id'] = df['firm_id'].map(lambda s: firm_ids_dict.get(s) if s in firm_ids_dict else s)
    
    return df

In [766]:
#calculate and melt
def create_df_quarterly_avgs(df, col, firm_ids_dict, firm_id):
    df = calculate_df_quarterly_avgs(df, 'term', firm_ids_dict)
    df = melt_avgs(df, firm_id, 'quarter')
    
    #lowercase quarter column
    df.quarter = df.quarter.str.lower()
    return df

In [771]:
#generate all quarterly average DataFrames
df_eps_fc_quarter_avgs = create_df_quarterly_avgs(eps_fc_clean, 'term', firm_ids_dict, 'firm_id')
df_eps_act_quarter_avgs = create_df_quarterly_avgs(eps_act_clean, 'term', firm_ids_dict, 'firm_id')
df_eod_act_quarter_avgs = create_df_quarterly_avgs(eod_act_clean, 'term', firm_ids_dict, 'firm_id')
df_eps_fc_terms_quarter_avgs = create_df_quarterly_avgs(eps_fc_terms_clean, 'term', firm_ids_dict, 'firm_id')

In [772]:
#create copies
eps_fc_quarter_avgs = df_eps_fc_quarter_avgs.copy()
eps_act_quarter_avgs = df_eps_act_quarter_avgs.copy()
eod_act_quarter_avgs = df_eod_act_quarter_avgs.copy()
eps_fc_terms_quarter_avgs = df_eps_fc_terms_quarter_avgs.copy()

In [773]:
#add new column entailing "type" of feature before concat
eps_fc_quarter_avgs = eps_fc_quarter_avgs.assign(feature = 'eps_fc')
eps_act_quarter_avgs = eps_act_quarter_avgs.assign(feature = 'eps_act')
eod_act_quarter_avgs = eod_act_quarter_avgs.assign(feature = 'eod_act')
eps_fc_terms_quarter_avgs = eps_fc_terms_quarter_avgs.assign(feature = 'eps_fc_terms')

In [774]:
#put DataFrames in array
quarterly_avgs_arr = [eps_fc_quarter_avgs,
                  eps_act_quarter_avgs,
                  eod_act_quarter_avgs,
                  eps_fc_terms_quarter_avgs]

In [775]:
#merge all quarterly average DataFrames
df_quarter_avgs = pd.concat(quarterly_avgs_arr)

### Test

In [776]:
#each quarter individually

#eps_fc_quarter
eps_fc_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
1531,16,q4,0.486857,eps_fc
81,81,q1,0.729667,eps_fc
700,195,q2,NaN,eps_fc
1900,385,q4,1.526625,eps_fc
801,296,q2,0.457095,eps_fc
1711,196,q4,0.841900,eps_fc
1023,13,q3,2.195947,eps_fc
605,100,q2,0.381476,eps_fc
974,469,q2,1.216952,eps_fc
1565,50,q4,0.295750,eps_fc


In [777]:
#eps_act_quarter
eps_act_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
629,124,q2,0.542381,eps_act
731,226,q2,2.253000,eps_act
141,141,q1,0.246786,eps_act
1296,286,q3,1.188810,eps_act
377,377,q1,1.438095,eps_act
1099,89,q3,0.136190,eps_act
1849,334,q4,-0.004500,eps_act
1469,459,q3,0.464286,eps_act
1811,296,q4,0.391667,eps_act
419,419,q1,0.760629,eps_act


In [778]:
#eod_act_quarter
eod_act_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
1226,216,q3,30.850895,eod_act
1046,36,q3,95.177524,eod_act
1097,87,q3,53.798421,eod_act
532,27,q2,25.312562,eod_act
665,160,q2,42.903333,eod_act
1995,480,q4,78.044000,eod_act
506,1,q2,38.682500,eod_act
185,185,q1,39.162976,eod_act
132,132,q1,28.812500,eod_act
1161,151,q3,61.184581,eod_act


In [779]:
#eps_fc_terms_quarter
eps_fc_terms_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
1993,478,q4,1.367800,eps_fc_terms
1214,204,q3,0.470750,eps_fc_terms
2006,491,q4,0.392692,eps_fc_terms
1389,379,q3,1.947950,eps_fc_terms
996,491,q2,0.384615,eps_fc_terms
1829,314,q4,1.737000,eps_fc_terms
707,202,q2,0.381600,eps_fc_terms
1252,242,q3,0.369200,eps_fc_terms
673,168,q2,0.667333,eps_fc_terms
843,338,q2,1.111000,eps_fc_terms


In [780]:
#verify that all columns are consistent
print(eps_fc_quarter_avgs.columns, '\n',
      eps_act_quarter_avgs.columns, '\n',
      eod_act_quarter_avgs.columns, '\n',
      eps_fc_terms_quarter_avgs.columns)

Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object')


In [781]:
#verify there are 505 firms in each dataframe
print(eps_fc_quarter_avgs.firm_id.nunique(), '\n',
      eps_act_quarter_avgs.firm_id.nunique(), '\n',
      eod_act_quarter_avgs.firm_id.nunique(), '\n',
      eps_fc_terms_quarter_avgs.firm_id.nunique())

505 
 505 
 505 
 505


In [782]:
#all quarterly averages
df_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
157,157,q1,0.436905,eps_act
1283,273,q3,0.748571,eps_act
1595,80,q4,0.188018,eps_act
230,230,q1,65.045762,eod_act
1402,392,q3,2.073450,eps_fc_terms
369,369,q1,63.262562,eod_act
1275,265,q3,0.335200,eps_fc
1315,305,q3,0.386667,eps_act
1067,57,q3,0.927381,eps_fc
1665,150,q4,58.352125,eod_act


In [783]:
#verify 505 firms in df_quarter_avgs
df_quarter_avgs.firm_id.nunique()

505

In [786]:
df_quarter_avgs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8080 entries, 0 to 2019
Data columns (total 4 columns):
firm_id    8080 non-null int64
quarter    8080 non-null object
average    8021 non-null float64
feature    8080 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 315.6+ KB


### Code
>**ISSUE 10:** Unnormalized data among `df_twenty_year_avgs`, `df_yearly_avgs`, and `df_quarter_avgs`

**Define:** 
- Normalize features across all 3 DataFrames. 
- For `df_twenty_year_avgs`, melt **feature column names** (eps_fc, eps_act, eod_act, eps_fc_terms) into a single column called **feature**

Desired output for each DataFrame:


| firm_id     | average  | average_type | time_period   | feature|
|:------------|:-------------:|:-----------:|:---------:|-------:|
|501           | 0.21875         | yearly   | 2002        | eps_fc_terms |
| 378          |   0.059000     |   quarterly | q4        | eps_act  |
| 147           | NaN     | yearly   |   2016      | eod_act  |
| 33           |  0.005067       | quarterly  | q4      | eps_fc |

> ***Apply the above table format to all three DFs, then concatenate to create `all_avgs` .***

**Note:** For `df_twenty_year_avgs`, it would be redundant to put *twenty_year* as the value under both **average_type** and **time_period** columns.

To curb this, I will assign all **time_period** values as NaN for `df_twenty_year_avgs`.

In [788]:
#assign column order
avgs_col_order = ['firm_id', 'average', 'average_type', 'time_period', 'feature']

In [789]:
#create clean copies of dfs
twenty_year_clean = df_twenty_year_avgs.copy()
yearly_clean = df_yearly_avgs.copy()
quarterly_clean = df_quarter_avgs.copy()

- ### `twenty_year_clean`

In [790]:
#isolate feature names
feats_col_names = twenty_year_clean.columns.difference(['firm_id'])

In [791]:
#move eps_fc . . . eps_fc_terms under "average_type"
twenty_year_clean = pd.melt(twenty_year_clean,
        id_vars = ['firm_id'],
        value_vars = feats_col_names,
        var_name = 'feature',
        value_name = 'average')

In [792]:
#add 'feature' and 'time_period' columns
twenty_year_clean = twenty_year_clean.assign(average_type = 'twenty_year', time_period = np.nan)

In [793]:
#reorder columns
twenty_year_clean = twenty_year_clean[avgs_col_order]

### Test

In [794]:
#twenty_year
twenty_year_clean.sample(5)

,firm_id,average,average_type,time_period,feature
219,219,14.245000,twenty_year,NaN,eod_act
208,208,836.688087,twenty_year,NaN,eod_act
610,105,0.844320,twenty_year,NaN,eps_act
26,26,56.687067,twenty_year,NaN,eod_act
1467,457,0.948289,twenty_year,NaN,eps_fc


In [799]:
twenty_year_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 5 columns):
firm_id         2020 non-null int64
average         2012 non-null float64
average_type    2020 non-null object
time_period     0 non-null float64
feature         2020 non-null object
dtypes: float64(2), int64(1), object(2)
memory usage: 79.0+ KB


In [795]:
#verify 505 firm names
twenty_year_clean.firm_id.nunique()

505

In [796]:
#verify all time_period values are NaN
twenty_year_clean.time_period.unique()

array([nan])

In [797]:
#verify average_type contains only twenty_year
twenty_year_clean.average_type.unique()

array(['twenty_year'], dtype=object)

- ### `yearly_clean`

In [798]:
#assign 'average_type' column
yearly_clean = yearly_clean.assign(average_type = 'yearly')

In [800]:
#rename 'year' column to 'time_period'
yearly_clean.rename(columns = {'year' : 'time_period'}, inplace = True)

In [801]:
#reorder columns
yearly_clean = yearly_clean[avgs_col_order]

### Test

In [803]:
yearly_clean.sample(10)

,firm_id,average,average_type,time_period,feature
10408,308,0.02000,yearly,2019,eps_act
3951,416,42.68250,yearly,2006,eod_act
6197,137,56.06000,yearly,2011,eod_act
9281,191,0.44275,yearly,2018,eps_fc_terms
2988,463,0.54150,yearly,2004,eps_fc
1505,495,0.57750,yearly,2001,eps_act
10018,423,1.78100,yearly,2019,eps_fc_terms
2699,174,0.70575,yearly,2004,eps_fc
10385,285,1.27500,yearly,2019,eps_fc
77,77,0.15575,yearly,1999,eps_fc


In [804]:
yearly_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41915 entries, 0 to 10099
Data columns (total 5 columns):
firm_id         41915 non-null int64
average         35971 non-null float64
average_type    41915 non-null object
time_period     41915 non-null object
feature         41915 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 1.9+ MB


In [805]:
#verify 505 firms
yearly_clean.firm_id.nunique()

505

In [806]:
#verify 'average_type' contains only 'yearly'
yearly_clean.average_type.unique()

array(['yearly'], dtype=object)

In [807]:
#verify 'feature' contains all features
all(feat in feats_col_names for feat in yearly_clean.feature.unique())

True

In [808]:
#verify there are 21 years recorded (1999 - 2019)
len(yearly_clean.time_period.unique())

21

In [809]:
#examine lowest and highest year
yearly_clean.time_period.min(), yearly_clean.time_period.max()

(1999, 2019)

- ### `quarterly_clean`

In [810]:
#assign 'average_type' column
quarterly_clean = quarterly_clean.assign(average_type = 'quarterly')

In [811]:
#rename 'quarter' column to 'time_period'
quarterly_clean.rename(columns = {'quarter' : 'time_period'}, inplace = True)

In [812]:
#reorder columns
quarterly_clean = quarterly_clean[avgs_col_order]

### Test

In [813]:
quarterly_clean.sample(10)

,firm_id,average,average_type,time_period,feature
1906,391,0.937000,quarterly,q4,eps_fc_terms
1388,378,0.417143,quarterly,q3,eps_act
360,360,1.097524,quarterly,q1,eps_fc
2014,499,1.145000,quarterly,q4,eps_fc
1366,356,0.255952,quarterly,q3,eps_act
1037,27,1.222733,quarterly,q3,eps_fc
305,305,0.256095,quarterly,q1,eps_fc
510,5,41.306738,quarterly,q2,eod_act
449,449,0.105200,quarterly,q1,eps_fc_terms
770,265,0.311000,quarterly,q2,eps_fc


In [815]:
quarterly_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8080 entries, 0 to 2019
Data columns (total 5 columns):
firm_id         8080 non-null int64
average         8021 non-null float64
average_type    8080 non-null object
time_period     8080 non-null object
feature         8080 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 378.8+ KB


In [816]:
#verify 505 firms
quarterly_clean.firm_id.nunique()

505

In [817]:
#verify 'average_type' contains only 'quarterly'
quarterly_clean.average_type.unique()

array(['quarterly'], dtype=object)

In [818]:
#verify 'time_period' contains only 'q1' till 'q4'
quarterly_clean.time_period.unique()

array(['q1', 'q2', 'q3', 'q4'], dtype=object)

In [819]:
#verify 'feature' contains all features
all(feat in feats_col_names for feat in quarterly_clean.feature.unique())

True

### Code
>**ISSUE 13:** - 20-year, yearly, and quarterly averages contained in different DataFrames.

**Define:** Concatenate the normalized `twenty_year_clean`, `yearly_clean`, and `quarterly_clean` into new CSV **all_avgs.csv**.

In [820]:
#put all average DFs in an array
avgs_array = [twenty_year_clean, yearly_clean, quarterly_clean]

In [821]:
all_avgs = pd.concat(avgs_array)

### Test

In [822]:
all_avgs.sample(10)

,firm_id,average,average_type,time_period,feature
2850,325,NaN,yearly,2004,eps_fc
9937,342,76.245000,yearly,2018,eod_act
9687,92,1.415000,yearly,2018,eps_act
10362,262,1.172000,yearly,2019,eps_fc
4213,173,0.123125,yearly,2007,eps_act
2163,143,30.553650,yearly,2003,eod_act
1300,290,0.682000,yearly,2002,eps_fc_terms
6670,105,0.827250,yearly,2013,eps_fc_terms
2829,304,0.288750,yearly,2004,eps_fc
7354,284,31.490000,yearly,2013,eod_act


In [824]:
all_avgs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52015 entries, 0 to 2019
Data columns (total 5 columns):
firm_id         52015 non-null int64
average         46004 non-null float64
average_type    52015 non-null object
time_period     49995 non-null object
feature         52015 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 2.4+ MB


In [825]:
all_avgs.shape

(52015, 5)

In [826]:
#verify 505 firms, no duplicates
all_avgs.firm_id.nunique()

505

In [827]:
#verify only 3 average_types
all_avgs.average_type.unique()

array(['twenty_year', 'yearly', 'quarterly'], dtype=object)

In [828]:
#verify 4 features
all_avgs.feature.unique()

array(['eod_act', 'eps_act', 'eps_fc', 'eps_fc_terms'], dtype=object)

### Code
>**ISSUE 14:** Unnormalized data among `eps_act`, `eps_fc`, `eod_act`, and `eps_fc_terms`

**Define:**

- Achieve desired `features.csv` output:

| firm_id  | term_forecast     | value      | feature         |    date    |
|:---------| :-------: | :-----------------:| :-------------: |----------: |
| 44       |     2004Q1  |  3.55           |        eps_fc      |  NaN      |
| 55       | 2004Q4     |  6.22       |  eps_fc_terms       | 2004-07-01 |
| 202      | 2015Q1      | 6.28      |   eod_act         |   1999-03-31    |


> **date** feature contains the **date column in `eod_act`** and **forecast_made in `eps_fc_terms`**. They are assigned to the same column because both columns are consistent: the same DateTime Object.

> **date** feature will be NaN for `eps_fc` and `eps_act`.

- ### `eps_fc` and `eps_act`

> For both DataFrames, we don't have to worry about handling the **date** column. That will be resolved later when merging all the DataFrames with `eod_act` and `eps_fc_terms`

In [829]:
fc_act_order = ['firm_id', 'feature', 'term', 'value']

In [830]:
#melt firm names
eps_fc_clean = eps_fc_clean.melt (id_vars = ['term'],
                  var_name = 'firm_id')

In [831]:
eps_act_clean = eps_act_clean.melt(id_vars = ['term'],
                                  var_name = 'firm_id')

In [832]:
#reassign all firms to firm_id
eps_fc_clean['firm_id'] = eps_fc_clean['firm_id'].map(firm_ids_dict)
eps_act_clean['firm_id'] = eps_act_clean['firm_id'].map(firm_ids_dict)

In [833]:
#add feature column
eps_fc_clean = eps_fc_clean.assign(feature = 'eps_fc')
eps_act_clean = eps_act_clean.assign(feature = 'eps_act')

In [834]:
#reorder columns
eps_fc_clean = eps_fc_clean[fc_act_order]
eps_act_clean = eps_act_clean[fc_act_order]

### Test

In [835]:
eps_fc_clean.sample(10)

,firm_id,feature,term,value
11871,141,eps_fc,2005Q4,0.045
14223,169,eps_fc,2005Q4,2.850
35274,419,eps_fc,2018Q3,1.802
35329,420,eps_fc,2011Q2,0.663
34420,409,eps_fc,2015Q1,0.231
32327,384,eps_fc,2016Q4,1.885
12689,151,eps_fc,2000Q2,1.249
18527,220,eps_fc,2010Q4,NaN
6044,71,eps_fc,2019Q1,6.130
7781,92,eps_fc,2012Q2,0.593


In [836]:
eps_fc_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42420 entries, 0 to 42419
Data columns (total 4 columns):
firm_id    42420 non-null int64
feature    42420 non-null object
term       42420 non-null period[Q-DEC]
value      35365 non-null float64
dtypes: float64(1), int64(1), object(1), period[Q-DEC](1)
memory usage: 1.3+ MB


In [837]:
#verify 505 firms in both
eps_fc_clean.firm_id.nunique() == 505 & eps_act_clean.firm_id.nunique() == 505

True

In [838]:
#examine 'feature' in both
len(eps_fc_clean.feature.unique()) == 1 & len(eps_act_clean.feature.unique()) == 1

True

In [839]:
#verify 84 unique term values: 21 years x 4 quarters
len(eps_fc_clean.term.unique()), len(eps_act_clean.term.unique())

(84, 84)

- ### `eod_act`

In [840]:
#define column order
feats_col_order = ['firm_id', 'feature', 'date', 'term', 'value']

In [841]:
#melt firm ids
eod_act_clean = eod_act_clean.melt(id_vars = ['date', 'term'],
                  var_name = 'firm_id')

In [842]:
#reassign firm_ids
eod_act_clean['firm_id'] = eod_act_clean['firm_id'].map(firm_ids_dict)

In [843]:
#add feature column
eod_act_clean = eod_act_clean.assign(feature = 'eod_act')

In [844]:
#reorder columns 
eod_act_clean = eod_act_clean[feats_col_order]

### Test

In [845]:
eod_act_clean.sample(5)

,firm_id,feature,date,term,value
25961,309,eod_act,2000-06-30,2000Q2,10.7776
27744,330,eod_act,2005-03-31,2005Q1,102.0600
39900,475,eod_act,1999-03-31,1999Q1,5.2546
40873,486,eod_act,2011-06-30,2011Q2,NaN
31500,375,eod_act,1999-03-31,1999Q1,20.7500


In [846]:
eod_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42420 entries, 0 to 42419
Data columns (total 5 columns):
firm_id    42420 non-null int64
feature    42420 non-null object
date       42420 non-null datetime64[ns]
term       42420 non-null period[Q-DEC]
value      35499 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(1), period[Q-DEC](1)
memory usage: 1.6+ MB


In [847]:
#verify 505 firms 
eod_act_clean.firm_id.nunique() == 505 

True

In [848]:
#examine 'feature' column
eod_act_clean.feature.unique()

array(['eod_act'], dtype=object)

In [849]:
#verify 84 unique dates and terms
len(eod_act_clean.term.unique()) == 84, len(eod_act_clean.date.unique()) == 84

(True, True)

- ### `eps_fc_terms`

In [851]:
#melt firm ids
eps_fc_terms_clean = eps_fc_terms_clean.melt(id_vars = ['forecast_made', 'term'],
                       var_name = 'firm_id')

In [852]:
#reassign firm ids
eps_fc_terms_clean['firm_id'] = eps_fc_terms_clean['firm_id'].map(firm_ids_dict)

In [853]:
#rename forecast_made to date
eps_fc_terms_clean = eps_fc_terms_clean.rename(columns = {'forecast_made' : 'date'})

In [854]:
#add feature column
eps_fc_terms_clean = eps_fc_terms_clean.assign(feature = 'eps_fc_terms')

In [855]:
#reorder columns
eps_fc_terms_clean = eps_fc_terms_clean[feats_col_order]

### Test

In [861]:
eps_fc_terms_clean.sample(10)

,firm_id,feature,date,term,value
38020,475,eps_fc_terms,2004-10-01,2005Q1,NaN
17950,224,eps_fc_terms,2007-04-01,2007Q3,0.550
38821,485,eps_fc_terms,2005-01-01,2005Q2,1.463
34478,430,eps_fc_terms,2019-04-01,2019Q3,0.390
3766,47,eps_fc_terms,2001-04-01,2001Q3,0.085
15874,198,eps_fc_terms,2008-04-01,2008Q3,0.375
14642,183,eps_fc_terms,2000-04-01,2000Q3,0.130
25836,322,eps_fc_terms,2018-10-01,2019Q1,0.423
20076,250,eps_fc_terms,2018-10-01,2019Q1,1.208
39599,494,eps_fc_terms,2019-07-01,2019Q4,1.845


In [863]:
#verify 505 firms
eps_fc_terms_clean.firm_id.nunique()

505

In [864]:
#examine 'feature' column
eps_fc_terms_clean.feature.unique()

array(['eps_fc_terms'], dtype=object)

In [865]:
#verify 80 unique terms (1999 not recorded)
eps_fc_terms_clean.term.nunique() == 80

True

- ### `features.csv`

Now's the time to bring everything together!

In [866]:
feat_array = [eps_fc_clean, eps_act_clean, eod_act_clean, eps_fc_terms_clean]

In [867]:
#concat to prepare for outer left merge
eod_act_eps_terms = pd.concat([eod_act_clean, eps_fc_terms_clean], join = 'outer')

In [868]:
#outer join
features = pd.concat([eod_act_eps_terms, eps_fc_clean], join = 'outer', sort = False)
features = pd.concat([features, eps_act_clean], join = 'outer', sort = False)

In [869]:
features.sample(10)

,firm_id,feature,date,term,value
23282,277,eps_fc,NaT,2002Q3,0.239
28243,336,eps_fc,NaT,2003Q4,NaN
31117,370,eod_act,2008-06-30,2008Q2,18.720
14364,171,eps_fc,NaT,1999Q1,0.182
31303,372,eod_act,2012-12-31,2012Q4,18.160
10057,125,eps_fc_terms,2014-01-01,2014Q2,0.631
17780,211,eps_act,NaT,2013Q1,0.295
28292,336,eps_act,NaT,2016Q1,0.320
33621,400,eod_act,2004-06-30,2004Q2,12.740
16839,210,eps_fc_terms,2009-07-01,2009Q4,0.559


### Test

In [870]:
features.sample(5)

,firm_id,feature,date,term,value
9038,107,eps_act,NaT,2011Q3,2.350
18640,221,eps_act,NaT,2018Q1,NaN
36361,432,eps_fc,NaT,2017Q2,0.736
15765,197,eps_fc_terms,2001-01-01,2001Q2,NaN
22130,263,eps_fc,NaT,2008Q3,0.335


In [872]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167660 entries, 0 to 42419
Data columns (total 5 columns):
firm_id    167660 non-null int64
feature    167660 non-null object
date       82820 non-null datetime64[ns]
term       167660 non-null period[Q-DEC]
value      141079 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(1), period[Q-DEC](1)
memory usage: 7.7+ MB


In [873]:
#verify 505 terms
features.firm_id.nunique() == 505

True

In [874]:
#examine 'feature' column
features.feature.unique()

array(['eod_act', 'eps_fc_terms', 'eps_fc', 'eps_act'], dtype=object)

In [875]:
#verify 84 term ranges
features.term.nunique() == 84, 

(True,)

In [876]:
#verify 164 date values: 84 ones from eod_act, and 80 from eps_fc_terms
features.date.nunique()

164

# III) Store Data <a id = "store"></a>

**Disclaimer:** I decided to put the following CSVs under the ***./data/clean/averages/components*** directory because they form *part* of the main CSVs under the ***./data/clean/averages*** directory:
- `quarter-avgs-eod-act.csv`
- `quarter-avgs-eps-act.csv`
- `quarter-avgs-eps-fc.csv`
- `quarter-avgs-eps-fc-terms.csv`

> The above are all components of the `quarter-avgs.csv` under **/averages/**

- `yearly-avgs-eod-act.csv`
- `yearly-avgs-eps-act.csv`
- `yearly-avgs-eps-fc.csv`
- `yearly-avgs-eps-fc-terms.csv`

> The above are all components of the `yearly-avgs.csv` under **/averages/**

**Additionally, I decided to keep `avgs.csv` under the *./data/clean/averages/* directory.** Although this CSV containing all averages would be highly inefficient for the data exploration stage, I decided it would be best to keep it on hand for convenience.

Using `avgs.csv` would mean having the parse through the column names to isolate 20-year, yearly, and quarterly averages, but all of that redundance can be avoided by simply focusing on the `twenty-year-avgs`, `yearly-avgs`, and `quarter-avgs` CSVs alone.

## Firms

In [877]:
firm_ids.to_csv(PATH_CLEAN + 'firms.csv', encoding = 'utf-8', index = False)

## Averages

In [878]:
#twenty-year-averages
df_twenty_year_avgs.to_csv(PATH_CLEAN_AVGS + 'twenty-year-avgs.csv', encoding = 'utf-8', index = False)

In [879]:
#yearly averagas
df_yearly_avgs.to_csv(PATH_CLEAN_AVGS + 'yearly-avgs.csv', encoding = 'utf-8', index = False)

In [880]:
#quarterly averages
df_quarter_avgs.to_csv(PATH_CLEAN_AVGS + 'quarter-avgs.csv', encoding = 'utf-8', index = False)

In [881]:
#all averages
all_avgs.to_csv(PATH_CLEAN_AVGS + 'avgs.csv', encoding = 'utf-8', index = False)

**NOTE:** For the twenty year, yearly, and quarterly datasets, I stored their old `df_` version instead of their new clean versions. 
> This is because their old versions are stored more efficiently as ***standalone*** CSVs, and were only modified to fit the larger `all_avgs` DataFrame.

## Average Components

In [882]:
#quarterly averages
store_df(PATH_CLEAN_AVGS_SUB, df_eps_fc_quarter_avgs, 'quarter-avgs-eps-fc.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eps_act_quarter_avgs, 'quarter-avgs-eps-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eod_act_quarter_avgs, 'quarter-avgs-eod-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eps_fc_terms_quarter_avgs, 'quarter-avgs-eps-fc-terms.csv')

In [883]:
#store yearly averages
store_df(PATH_CLEAN_AVGS_SUB, df_eps_fc_yearly_avgs, 'yearly-avgs-eps-fc.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eps_act_yearly_avgs, 'yearly-avgs-eps-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eod_act_yearly_avgs, 'yearly-avgs-eod-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eps_fc_terms_yearly_avgs, 'yearly-avgs-eps-fc-terms.csv')

## Features

In [884]:
eps_fc_clean.to_csv(PATH_CLEAN + 'eps-fc.csv')
eps_act_clean.to_csv(PATH_CLEAN + 'eps-act.csv')
eod_act_clean.to_csv(PATH_CLEAN + 'eod-act.csv')
eps_fc_terms_clean.to_csv(PATH_CLEAN + 'eps-fc-terms.csv')

features.to_csv(PATH_CLEAN + 'features.csv')

# IV) Notes



- When dealing with the CSVs depicting averages, we need to keep in mind **missing data** in our interpretations.
- I decided to put the years as columns under yearly average DF generation because: the amount of columns generated here, TK, would be less than the amount of 
	firms there are, 505. [TK link to a study showing how having too many columns take up memory vs rows]
    
- keep in mind that `eod_act` is based on **calendar years** not fiscal years.

In [136]:

#convert notebook to HTML